<a href="https://colab.research.google.com/github/paulmaegoh/wyhow/blob/master/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd


Import Libraries

#Select target page